<a href="https://colab.research.google.com/github/harnalashok/deeplearning/blob/main/simpleNN_churn_modeling_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 29th October, 2022
# California Housing Dataset
# Experiments with NN
# https://www.kaggle.com/datasets/camnugent/california-housing-prices

In [24]:
# 1.0
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

# 1.1
import tensorflow as tf

# 1.2 Helper libraries
import numpy as np
import matplotlib.pyplot as plt


In [27]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [3]:
# 2.0 Mount google drive 
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
# 2.1 Read Bank Customers churn data

path = "/gdrive/MyDrive/Colab_data_files/bank_customers_churn/"


In [6]:
# 2.2
data = pd.read_csv(path + "Churn_Modelling.csv")

In [7]:
# 2.2.1
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
# 2.2.2
data.shape   # (10000, 14)

(10000, 14)

In [9]:
# 2.3 Pop out target
y = data.pop('Exited')

In [10]:
# 2.3.1
data.shape   # (10000,13)

(10000, 13)

In [11]:
# 3.0 Check NULL
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

In [13]:
data['Gender'] = data['Gender'].map({'Female' : 1, 'Male' : 0})

In [14]:
data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [15]:
data['Geography'] = data['Geography'].map({'France' : 0, 'Spain' : 2, 'Germany' : 3})

In [16]:
# 3.2 Categorical features:
data[:2]

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1,15634602,Hargrave,619,0,1,42,2,0.00,1,1,1,101348.88
1,2,15647311,Hill,608,2,1,41,1,83807.86,1,0,1,112542.58


In [19]:
data['CustomerId'].duplicated().sum()

0

In [21]:
data = data.drop(columns = ['RowNumber','CustomerId', 'Surname'])

In [22]:
data['Tenure'].unique()

array([ 2,  1,  8,  7,  4,  6,  3, 10,  5,  9,  0])

In [25]:
X_train,X_test, y_train,y_test = train_test_split(data, y, test_size = 0.25)

In [28]:
X_train.shape
X_test.shape

(7500, 10)

(2500, 10)

In [29]:
mm = MinMaxScaler()
mm.fit(X_train)

MinMaxScaler()

In [30]:
X_train = mm.transform(X_train)
X_test = mm.transform(X_test)

## Model

In [61]:
# 6.0
model = tf.keras.Sequential()

In [62]:
# 6.1
model.add( tf.keras.layers.Input(shape = (10,) ))

In [63]:
# 6.2 Start 
model.add(tf.keras.layers.Dense(40, activation = 'relu'))  # MAke it 5 and then 20 (not more or less)

In [64]:
# 6.2.1 Experiment with adding a dropout layer
#       but then increase number of units in Dense layer from 20 to 40
#model.add(tf.keras.layers.Dropout(rate = 0.5 ))

In [65]:
model.add(tf.keras.layers.Dense(20, activation = 'relu'))  
#model.add(tf.keras.layers.Dropout(rate = 0.5 ))

In [66]:
model.add(tf.keras.layers.Dense(10, activation = 'relu'))  
#model.add(tf.keras.layers.Dropout(rate = 0.5 ))

In [67]:
# 6.3 Experiment first with activation of sigmoid
#     and then no activation function
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))    # Keep sigmoid; then remove sigmoid

In [68]:
# 6.4 Model summary:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 40)                440       
                                                                 
 dense_13 (Dense)            (None, 20)                820       
                                                                 
 dense_14 (Dense)            (None, 10)                210       
                                                                 
 dense_15 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,481
Trainable params: 1,481
Non-trainable params: 0
_________________________________________________________________


In [69]:
# 6.5 Compile model
#     Expt with adam
model.compile(
                 loss = 'binary_crossentropy',
                 optimizer = 'adam',  # Try first with default optimizer and then with 'adam'
                                      #  may not make much difference 
                 metrics = ['acc']

              )

In [70]:
# 7.0
history = model.fit(X_train,y_train,
                    epochs = 70,
                    validation_data = (X_test, y_test)
                   )

Epoch 1/70
235/235 [==============================] - 1s 3ms/step - loss: 0.5015 - acc: 0.7916 - val_loss: 0.4396 - val_acc: 0.8100
Epoch 2/70
235/235 [==============================] - 1s 3ms/step - loss: 0.4526 - acc: 0.7927 - val_loss: 0.3981 - val_acc: 0.8232
Epoch 3/70
235/235 [==============================] - 1s 3ms/step - loss: 0.4085 - acc: 0.8223 - val_loss: 0.3671 - val_acc: 0.8452
Epoch 4/70
235/235 [==============================] - 1s 2ms/step - loss: 0.3816 - acc: 0.8399 - val_loss: 0.3553 - val_acc: 0.8556
Epoch 5/70
235/235 [==============================] - 1s 3ms/step - loss: 0.3695 - acc: 0.8463 - val_loss: 0.3477 - val_acc: 0.8644
Epoch 6/70
235/235 [==============================] - 1s 3ms/step - loss: 0.3657 - acc: 0.8477 - val_loss: 0.3497 - val_acc: 0.8596
Epoch 7/70
235/235 [==============================] - 1s 3ms/step - loss: 0.3617 - acc: 0.8493 - val_loss: 0.3442 - val_acc: 0.8660
Epoch 8/70
235/235 [==============================] - 1s 3ms/step - loss: 0.

In [ ]:
# 7.1
model.evaluate(X_test,y_test)

97/97 [==============================] - 0s 2ms/step - loss: 0.6292 - mae: 0.6292


[0.6291962265968323, 0.6291962265968323]

In [ ]:
######################### I am done #################